# Market Insights Analysis

## Libraries Importing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import os
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster, cut_tree
from scipy.spatial.distance import pdist
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, ConfusionMatrixDisplay, RocCurveDisplay, confusion_matrix
from sklearn.metrics import silhouette_score

from datetime import timedelta
from lifetimes.utils import summary_data_from_transaction_data, calibration_and_holdout_data
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix, plot_calibration_purchases_vs_holdout_purchases

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from yellowbrick.cluster import KElbowVisualizer


import s3fs
import json
from sklearn.metrics.pairwise import cosine_similarity
import datetime as dt

from xgboost import XGBClassifier

import imblearn

import warnings
warnings.filterwarnings('ignore')


## Data Importing

In [2]:
sales = pd.read_csv('Data/Online_Sales.csv')
products = pd.read_csv('Data/products.csv') # extract unique product ID/name/group from table 'sales'
products.set_index('Product_SKU', inplace=True)
marketing = pd.read_csv('Data/Marketing_Spend.csv')
customer = pd.read_csv('Data/CustomersData.csv')
coupon = pd.read_csv('Data/Discount_Coupon.csv')
tax = pd.read_csv('Data/Tax_amount.csv')


## Data Overview

In [3]:
# sales data preview
sales.head()

CustomerID  Transaction_ID Transaction_Date     Product_SKU  \
0       17850           16679           1/1/19  GGOENEBJ079499   
1       17850           16680           1/1/19  GGOENEBJ079499   
2       17850           16681           1/1/19  GGOEGFKQ020399   
3       17850           16682           1/1/19  GGOEGAAB010516   
4       17850           16682           1/1/19  GGOEGBJL013999   

                                 Product_Description Product_Category  \
0  Nest Learning Thermostat 3rd Gen-USA - Stainle...         Nest-USA   
1  Nest Learning Thermostat 3rd Gen-USA - Stainle...         Nest-USA   
2              Google Laptop and Cell Phone Stickers           Office   
3  Google Men's 100% Cotton Short Sleeve Hero Tee...          Apparel   
4                    Google Canvas Tote Natural/Navy             Bags   

   Quantity  Avg_Price  Delivery_Charges Coupon_Status  
0         1     153.71               6.5          Used  
1         1     153.71               6.5          Used  
2         1       2.05               6.5          Used  
3         5      17.53               6.5      Not Used  
4         1      16.50               6.5          Used

In [4]:
#sales['Coupon_Status'].unique().value_counts()
np.unique(sales['Coupon_Status'], return_counts=True)

(array(['Clicked', 'Not Used', 'Used'], dtype=object),
 array([26926,  8094, 17904]))

In [5]:
coupon.head()

Month Product_Category Coupon_Code  Discount_pct
0   Jan          Apparel      SALE10            10
1   Feb          Apparel      SALE20            20
2   Mar          Apparel      SALE30            30
3   Jan         Nest-USA      ELEC10            10
4   Feb         Nest-USA      ELEC20            20

In [6]:
# Summary of sales data
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           52924 non-null  int64  
 1   Transaction_ID       52924 non-null  int64  
 2   Transaction_Date     52924 non-null  object 
 3   Product_SKU          52924 non-null  object 
 4   Product_Description  52924 non-null  object 
 5   Product_Category     52924 non-null  object 
 6   Quantity             52924 non-null  int64  
 7   Avg_Price            52924 non-null  float64
 8   Delivery_Charges     52924 non-null  float64
 9   Coupon_Status        52924 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 4.0+ MB


In [7]:
# Count of rows for each product category in sales
sales['Product_Category'].value_counts()

Product_Category
Apparel                 18126
Nest-USA                14013
Office                   6513
Drinkware                3483
Lifestyle                3092
Nest                     2198
Bags                     1882
Headgear                  771
Notebooks & Journals      749
Waze                      554
Nest-Canada               317
Bottles                   268
Accessories               234
Fun                       160
Gift Cards                159
Housewares                122
Google                    105
Backpacks                  89
More Bags                  46
Android                    43
Name: count, dtype: int64

In [8]:
# Sales data statistics summary
sales.describe()

CustomerID  Transaction_ID      Quantity     Avg_Price  \
count  52924.00000    52924.000000  52924.000000  52924.000000   
mean   15346.70981    32409.825675      4.497638     52.237646   
std     1766.55602     8648.668977     20.104711     64.006882   
min    12346.00000    16679.000000      1.000000      0.390000   
25%    13869.00000    25384.000000      1.000000      5.700000   
50%    15311.00000    32625.500000      1.000000     16.990000   
75%    16996.25000    39126.250000      2.000000    102.130000   
max    18283.00000    48497.000000    900.000000    355.740000   

       Delivery_Charges  
count      52924.000000  
mean          10.517630  
std           19.475613  
min            0.000000  
25%            6.000000  
50%            6.000000  
75%            6.500000  
max          521.360000

In [9]:
# Products data preview
products.head()

Product_Category  \
Product_SKU                       
GGOENEBJ079499         Nest-USA   
GGOEGFKQ020399           Office   
GGOEGAAB010516          Apparel   
GGOEGBJL013999             Bags   
GGOEGBMJ013399             Bags   

                                              Product_Description  
Product_SKU                                                        
GGOENEBJ079499  Nest Learning Thermostat 3rd Gen-USA - Stainle...  
GGOEGFKQ020399              Google Laptop and Cell Phone Stickers  
GGOEGAAB010516  Google Men's 100% Cotton Short Sleeve Hero Tee...  
GGOEGBJL013999                    Google Canvas Tote Natural/Navy  
GGOEGBMJ013399                                          Sport Bag

In [10]:
# Marketing data preview
marketing.head()

Date  Offline_Spend  Online_Spend
0  1/1/19           4500       2424.50
1  1/2/19           4500       3480.36
2  1/3/19           4500       1576.38
3  1/4/19           4500       2928.55
4  1/5/19           4500       4055.30

In [11]:
# Summary of marketing data
marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           365 non-null    object 
 1   Offline_Spend  365 non-null    int64  
 2   Online_Spend   365 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 8.7+ KB


In [12]:
# Statistics summary of marketing data
marketing.describe()

Offline_Spend  Online_Spend
count     365.000000    365.000000
mean     2843.561644   1905.880740
std       952.292448    808.856853
min       500.000000    320.250000
25%      2500.000000   1258.600000
50%      3000.000000   1881.940000
75%      3500.000000   2435.120000
max      5000.000000   4556.930000

In [ ]:
marketing['Offline_Spend'].rename['Offline_Expenditure']

In [13]:
# Customer data previewing
customer.head()

CustomerID Gender    Location  Tenure_Months
0       17850      M     Chicago             12
1       13047      M  California             43
2       12583      M     Chicago             33
3       13748      F  California             30
4       15100      M  California             49

In [14]:
# Summary of customer data
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CustomerID     1468 non-null   int64 
 1   Gender         1468 non-null   object
 2   Location       1468 non-null   object
 3   Tenure_Months  1468 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 46.0+ KB


In [15]:
# Statistics summary of customer data
customer.describe()

CustomerID  Tenure_Months
count   1468.000000    1468.000000
mean   15314.386240      25.912125
std     1744.000367      13.959667
min    12346.000000       2.000000
25%    13830.500000      14.000000
50%    15300.000000      26.000000
75%    16882.250000      38.000000
max    18283.000000      50.000000

In [16]:
# Coupon data preview
coupon.head()

Month Product_Category Coupon_Code  Discount_pct
0   Jan          Apparel      SALE10            10
1   Feb          Apparel      SALE20            20
2   Mar          Apparel      SALE30            30
3   Jan         Nest-USA      ELEC10            10
4   Feb         Nest-USA      ELEC20            20

In [17]:
# Summary of coupon data
coupon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Month             204 non-null    object
 1   Product_Category  204 non-null    object
 2   Coupon_Code       204 non-null    object
 3   Discount_pct      204 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 6.5+ KB


In [18]:
# Statistics summary of coupon data
coupon.describe()

Discount_pct
count    204.000000
mean      20.000000
std        8.185052
min       10.000000
25%       10.000000
50%       20.000000
75%       30.000000
max       30.000000

In [19]:
# Tax data preview
tax.head()

Product_Category  GST
0         Nest-USA  10%
1           Office  10%
2          Apparel  18%
3             Bags  18%
4        Drinkware  18%

In [20]:
# Convert GST column percentage value to deceimal
def convert_percentage_to_decimal(percentage):
    # Remove '%' sign and convert the string to a decimal representation
    return float(percentage.replace('%', '')) / 100

# Apply the function to the 'GST' column in the DataFrame
tax['GST'] = tax['GST'].apply(convert_percentage_to_decimal)

In [21]:
# Summary of tax data
tax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_Category  20 non-null     object 
 1   GST               20 non-null     float64
dtypes: float64(1), object(1)
memory usage: 448.0+ bytes


In [22]:
# Statistics summary of tax data
tax.describe()

GST
count  20.000000
mean    0.116500
std     0.052443
min     0.050000
25%     0.087500
50%     0.100000
75%     0.180000
max     0.180000

## Data Cleaning

In [23]:
# Merge customer data to sales data on CustomerID
sales = sales.merge(customer, on='CustomerID')

In [24]:
# Change Transaction_Date column to datetime
sales['Transaction_Date'] = pd.to_datetime(sales['Transaction_Date'])

In [25]:
# Change Transaction_Date column to format month/date/year
sales['Transaction_Date'] = pd.to_datetime(sales['Transaction_Date'], format='%m/%d/%Y')

In [26]:
# Create new column 'Month' that extracts month abbreviation from Transaction_Date
sales['Month'] = sales['Transaction_Date'].dt.strftime('%b')

In [27]:
# Create a new column 'InvoiceMonthly' by converting 'Transaction_Date' to a monthly frequency period ('M'),
# then converting it back to 'datetime64[ns]' format to represent the start date of each month
sales['InvoiceMonthly'] = sales['Transaction_Date'].dt.to_period('M').dt.to_timestamp('M') #Convert back to 'datetime64[ns]'

# Create a new column 'InvoiceDaily' by converting 'Transaction_Date' to a daily frequency period ('D'),
# then converting it back to 'datetime64[ns]' format to retain the transaction dates at the daily level.
sales['InvoiceDaily'] = sales['Transaction_Date'].dt.to_period('D').dt.to_timestamp('D') #Convert back to 'datetime64[ns]'

In [28]:
# Change Date column to datetime
marketing['Date'] = pd.to_datetime(marketing['Date'])

In [29]:
# Change marketing's Date column to format month/date/year
marketing['Date'] = pd.to_datetime(marketing['Date'], format='%m/%d/%Y')
# Create new column 'Month' that extracts month abbreviation from Date
marketing['Month'] = marketing['Date'].dt.strftime('%b')
# Create new column 'total_spend' which sums offline_spend and online_spend
marketing['total_spend'] = marketing['Offline_Spend'] + marketing['Online_Spend']

In [30]:
# Left join sales and coupon data on Month and Product_Category
sales = pd.merge(sales, coupon, on=['Month', 'Product_Category'], how='left')
# Left join sales and tax data on Product_Category 
sales = pd.merge(sales, tax, on=['Product_Category'], how = 'left')

In [31]:
# Fill Discount_pct column with 0
sales['Discount_pct'].fillna(0, inplace=True)

In [32]:
# Calculate total revenue

# Define a function 'calculate_invoice' to compute the invoice amount
def calculate_invoice(row):
    # If 'Coupon_Status' is 'Used', it calculates the invoice with quantity, average price, discount percentage, GST, and delivery charges.
    if row['Coupon_Status'] == 'Used':
        invoice = ((row['Quantity'] * row['Avg_Price']) * (1 - row['Discount_pct']/100) * (1 + row['GST'])) + row['Delivery_Charges']
    else:
    # If 'Coupon_Status' is not 'Used', it calculates the invoice with quantity, average price, GST, and delivery charges.
        invoice = ((row['Quantity'] * row['Avg_Price']) * (1 + row['GST'])) + row['Delivery_Charges']
    return invoice

# Apply the 'calculate_invoice' function to each row of the 'sales' DataFrame using the apply function with axis=1.
# This computes the 'Revenue' column by calculating the invoice amount for each row.
sales['Revenue'] = sales.apply(lambda row: calculate_invoice(row), axis=1)

In [33]:
# Preview sales data to check the outcome of Revenue column
sales.head()

CustomerID  Transaction_ID Transaction_Date     Product_SKU  \
0       17850           16679       2019-01-01  GGOENEBJ079499   
1       17850           16680       2019-01-01  GGOENEBJ079499   
2       17850           16681       2019-01-01  GGOEGFKQ020399   
3       17850           16682       2019-01-01  GGOEGAAB010516   
4       17850           16682       2019-01-01  GGOEGBJL013999   

                                 Product_Description Product_Category  \
0  Nest Learning Thermostat 3rd Gen-USA - Stainle...         Nest-USA   
1  Nest Learning Thermostat 3rd Gen-USA - Stainle...         Nest-USA   
2              Google Laptop and Cell Phone Stickers           Office   
3  Google Men's 100% Cotton Short Sleeve Hero Tee...          Apparel   
4                    Google Canvas Tote Natural/Navy             Bags   

   Quantity  Avg_Price  Delivery_Charges Coupon_Status Gender Location  \
0         1     153.71               6.5          Used      M  Chicago   
1         1     153.71               6.5          Used      M  Chicago   
2         1       2.05               6.5          Used      M  Chicago   
3         5      17.53               6.5      Not Used      M  Chicago   
4         1      16.50               6.5          Used      M  Chicago   

   Tenure_Months Month InvoiceMonthly InvoiceDaily Coupon_Code  Discount_pct  \
0             12   Jan     2019-01-31   2019-01-01      ELEC10          10.0   
1             12   Jan     2019-01-31   2019-01-01      ELEC10          10.0   
2             12   Jan     2019-01-31   2019-01-01       OFF10          10.0   
3             12   Jan     2019-01-31   2019-01-01      SALE10          10.0   
4             12   Jan     2019-01-31   2019-01-01       AIO10          10.0   

    GST   Revenue  
0  0.10  158.6729  
1  0.10  158.6729  
2  0.10    8.5295  
3  0.18  109.9270  
4  0.18   24.0230

In [34]:
# Get unique product descriptions
unique_product_descriptions = sales['Product_Description'].unique()

# Sort unique product descriptions alphabetically
sorted_unique_product_descriptions = sorted(unique_product_descriptions)

# Print sorted unique product descriptions
print("Unique Product Descriptions (Sorted Alphabetically):")
for description in sorted_unique_product_descriptions:
    print(description)

Unique Product Descriptions (Sorted Alphabetically):
1 oz Hand Sanitizer
20 oz Stainless Steel Insulated Tumbler
22 oz Android Bottle
22 oz YouTube Bottle Infuser
23 oz Wide Mouth Sport Bottle
24 oz YouTube Sergeant Stripe Bottle
25L Classic Rucksack
26 oz Double Wall Insulated Bottle
7&quot; Dog Frisbee
8 pc Android Sticker Sheet
Android 17oz Stainless Steel Sport Bottle
Android 24 oz Contigo Bottle
Android 25 oz Green Apple Stainless Steel Bottle
Android 5-Panel Low Cap
Android BTTF Cosmos Graphic Tee
Android BTTF Moonshot Graphic Tee
Android Baby Esssentials Set
Android Glass Water Bottle with Black Sleeve
Android Hard Cover Journal
Android Heavyweight Long Sleeve Badge Tee Black
Android Infant Short Sleeve Tee Aqua
Android Infant Short Sleeve Tee Pewter
Android Infant Short Sleeve Tee Pink
Android Journal Book Set
Android Large Removable Sticker Sheet
Android Lifted Men's Short Sleeve Tee Blue
Android Luggage Tag
Android Lunch Kit
Android Matrix Tee White
Android Men's  Zip Hoodie


In [35]:
# Get unique product descriptions
unique_product_category = sales['Product_Category'].unique()

# Sort unique product descriptions alphabetically
sorted_unique_product_cateogry = sorted(unique_product_category)

# Print sorted unique product descriptions
print("Unique Product Categories (Sorted Alphabetically):")
for category in sorted_unique_product_cateogry:
    print(category)

Unique Product Categories (Sorted Alphabetically):
Accessories
Android
Apparel
Backpacks
Bags
Bottles
Drinkware
Fun
Gift Cards
Google
Headgear
Housewares
Lifestyle
More Bags
Nest
Nest-Canada
Nest-USA
Notebooks & Journals
Office
Waze


In [36]:
# List of unique product categories
unique_categories = sorted(sales['Product_Category'].unique())

# Iterate through each category and print unique product descriptions
for category in unique_categories:
    # Get unique product descriptions for the current category
    unique_products_category = sales[sales['Product_Category'] == category]['Product_Description'].unique()
    
    # Sort unique product descriptions alphabetically
    sorted_unique_products_category = sorted(unique_products_category)
    
    # Print category header
    print(f"\nUnique Products in Category '{category}' (Sorted Alphabetically):")
    
    # Print sorted unique product descriptions for the current category
    for product in sorted_unique_products_category:
        print(product)


Unique Products in Category 'Accessories' (Sorted Alphabetically):
Android Large Removable Sticker Sheet
Android Small Removable Sticker Sheet
Emoji Sticker Sheet
Google Emoji Sticker Pack
UpCycled Bike Saddle Bag
UpCycled Handlebar Bag
Waze Pack of 9 Decal Set

Unique Products in Category 'Android' (Sorted Alphabetically):
Android Men's Paradise Short Sleeve Tee Olive

Unique Products in Category 'Apparel' (Sorted Alphabetically):
Android 5-Panel Low Cap
Android BTTF Cosmos Graphic Tee
Android BTTF Moonshot Graphic Tee
Android Baby Esssentials Set
Android Heavyweight Long Sleeve Badge Tee Black
Android Infant Short Sleeve Tee Aqua
Android Infant Short Sleeve Tee Pewter
Android Infant Short Sleeve Tee Pink
Android Lifted Men's Short Sleeve Tee Blue
Android Matrix Tee White
Android Men's  Zip Hoodie
Android Men's 3/4 Sleeve Raglan Henley Black
Android Men's Engineer Short Sleeve Tee Charcoal
Android Men's Long & Lean Badge Tee Charcoal
Android Men's Long Sleeve Badge Crew Tee Heather
A

In [37]:
def categorize_product(description):
    # Convert description to lowercase for case-insensitive matching
    description_lower = description.lower()
    
    # Check if any of the specific keywords are present in the description
    if 'android' in description_lower:
        return 'Android'
    elif any(keyword in description_lower for keyword in ['google', 'nest', 'waze']):
        return 'Google'
    elif 'youtube' in description_lower:
        return 'YouTube'
    else:
        return 'No-Brand'
        
# Apply the categorization function to create a new column 'Product_Group'
sales['Product_Group'] = sales['Product_Description'].apply(categorize_product)

In [38]:

def update_category(category):
    if category == 'Android':
        return 'Apparel'
    elif category == 'Bottles':
        return 'Drinkware'
    elif category == 'Fun':
        return 'Accessories'
    elif category == 'Google':
        return 'Drinkware'
    elif category == 'More Bags':
        return 'Bags'
    elif category == 'Nest-Canada':
        return 'Nest'
    elif category == 'Nest-USA':
        return 'Nest'
    else:
        return category

# Applying the function to 'Product_Category' column using 'apply'
sales['Product_Category'] = sales['Product_Category'].apply(update_category)

In [39]:
# Preview sales data to check Product_Group column outcome
sales.head()

CustomerID  Transaction_ID Transaction_Date     Product_SKU  \
0       17850           16679       2019-01-01  GGOENEBJ079499   
1       17850           16680       2019-01-01  GGOENEBJ079499   
2       17850           16681       2019-01-01  GGOEGFKQ020399   
3       17850           16682       2019-01-01  GGOEGAAB010516   
4       17850           16682       2019-01-01  GGOEGBJL013999   

                                 Product_Description Product_Category  \
0  Nest Learning Thermostat 3rd Gen-USA - Stainle...             Nest   
1  Nest Learning Thermostat 3rd Gen-USA - Stainle...             Nest   
2              Google Laptop and Cell Phone Stickers           Office   
3  Google Men's 100% Cotton Short Sleeve Hero Tee...          Apparel   
4                    Google Canvas Tote Natural/Navy             Bags   

   Quantity  Avg_Price  Delivery_Charges Coupon_Status  ... Location  \
0         1     153.71               6.5          Used  ...  Chicago   
1         1     153.71               6.5          Used  ...  Chicago   
2         1       2.05               6.5          Used  ...  Chicago   
3         5      17.53               6.5      Not Used  ...  Chicago   
4         1      16.50               6.5          Used  ...  Chicago   

  Tenure_Months  Month InvoiceMonthly InvoiceDaily Coupon_Code Discount_pct  \
0            12    Jan     2019-01-31   2019-01-01      ELEC10         10.0   
1            12    Jan     2019-01-31   2019-01-01      ELEC10         10.0   
2            12    Jan     2019-01-31   2019-01-01       OFF10         10.0   
3            12    Jan     2019-01-31   2019-01-01      SALE10         10.0   
4            12    Jan     2019-01-31   2019-01-01       AIO10         10.0   

    GST   Revenue  Product_Group  
0  0.10  158.6729         Google  
1  0.10  158.6729         Google  
2  0.10    8.5295         Google  
3  0.18  109.9270         Google  
4  0.18   24.0230         Google  

[5 rows x 21 columns]

In [40]:
sales.to_csv('sales_cleaned.csv')
marketing.to_csv('marketing_cleaned.csv')

## Customer Centric Analysis

In [41]:
total_quantity = sales.groupby(['CustomerID', 'Product_Category'])['Quantity'].sum().reset_index()


In [42]:
total_quantity.head()

CustomerID Product_Category  Quantity
0       12346          Apparel         1
1       12346           Office         2
2       12347      Accessories         1
3       12347          Apparel       231
4       12347             Bags         6

In [43]:
# Calculate total spending per customer
total_spending = sales.groupby('CustomerID')['Avg_Price'].sum().reset_index()

# Calculate total delivery charges per customer
delivery_charges = sales.groupby('CustomerID')['Delivery_Charges'].sum().reset_index()

# Calculate total number of transactions per customer
num_transactions = sales.groupby('CustomerID')['Transaction_ID'].count().reset_index()

# Calculate total quantity purchased per customer
total_quantity = sales.groupby('CustomerID')['Quantity'].sum().reset_index()

# Merge total spending, delivery charges, number of transactions, and total quantity DataFrames
customer_analysis = pd.merge(total_spending, delivery_charges, on='CustomerID')
customer_analysis = pd.merge(customer_analysis, num_transactions, on='CustomerID')
customer_analysis = pd.merge(customer_analysis, total_quantity, on='CustomerID')

# Extract location from each customer
unique_locations = sales.groupby('CustomerID')['Location'].first().reset_index()
# Merge unique_locations to customer analysis dataframe
customer_analysis = pd.merge(customer_analysis, unique_locations, on='CustomerID')

# Extract gender from each customer
unique_gender = sales.groupby('CustomerID')['Gender'].first().reset_index()
# Merge unique_gender to customer analysis dataframe
customer_analysis = pd.merge(customer_analysis, unique_gender, on='CustomerID')

# Extract tenure months for each customer
unique_tenure_months = sales.groupby('CustomerID')['Tenure_Months'].first().reset_index()
# Merge unique_tenure_months to customer_analysis dataframe
customer_analysis = pd.merge(customer_analysis, unique_tenure_months, on='CustomerID')

# Count of 'Used' coupons per customer and merge into 'customer_analysis' without duplicates
used_coupons_count = sales[sales['Coupon_Status'] == 'Used'].groupby('CustomerID')['Coupon_Status'].count().reset_index()
used_coupons_count.columns = ['CustomerID', 'Used_Coupon_Count']
customer_analysis = pd.merge(customer_analysis, used_coupons_count, on='CustomerID', how='left').fillna(0)

# Count of 'Not Used' coupons per customer and merge into 'customer_analysis' without duplicates
not_use_coupons_count = sales[sales['Coupon_Status'] == 'Not Used'].groupby('CustomerID')['Coupon_Status'].count().reset_index()
not_use_coupons_count.columns = ['CustomerID', 'Not_Use_Coupon_Count']
customer_analysis = pd.merge(customer_analysis, not_use_coupons_count, on='CustomerID', how='left').fillna(0)

# Count of 'Click' coupons per customer and merge into 'customer_analysis' without duplicates
click_coupons_count = sales[sales['Coupon_Status'] == 'Clicked'].groupby('CustomerID')['Coupon_Status'].count().reset_index()
click_coupons_count.columns = ['CustomerID', 'Clicked_Coupon_Count']
customer_analysis = pd.merge(customer_analysis, click_coupons_count, on='CustomerID', how='left').fillna(0)

# Extract revenue of each customer
revenue = sales.groupby('CustomerID')['Revenue'].sum().reset_index()
# Merge revenue to customer_analysis dataframe
customer_analysis = pd.merge(customer_analysis, revenue, on='CustomerID')

# Rename columns 
customer_analysis.columns = ['CustomerID', 'Total_Spending', 'Delivery_Charges', 'Num_Transactions', 'Total_Quantity', 'Location', 'Gender', 'Tenure_Months', 'Used_Coupon_Count', 'Not_Use_Coupon_Count', 'Clicked_Coupon_Count', 'Revenue']


In [44]:
# Preview customer_analysis
customer_analysis.head(10)

CustomerID  Total_Spending  Delivery_Charges  Num_Transactions  \
0       12346           25.49            150.00                 2   
1       12347         3746.16            665.14                60   
2       12348          336.52            197.15                23   
3       12350         1312.40            127.88                17   
4       12356         1244.83            637.49                36   
5       12359          235.41             96.00                16   
6       12370         3965.71            692.98                91   
7       12373          420.30            102.96                14   
8       12377         5060.61            771.68                77   
9       12383         3078.59            593.79                69   

   Total_Quantity    Location Gender  Tenure_Months  Used_Coupon_Count  \
0               3    New York      F             31                2.0   
1             342    New York      M             20               20.0   
2             209  California      M             39               11.0   
3              21  California      M             25                6.0   
4              56     Chicago      F             31               11.0   
5              86    New York      M             41                2.0   
6             613    New York      F             21               21.0   
7              22    New York      F             23                3.0   
8             421  California      F             27               21.0   
9             185    New York      M             32               21.0   

   Not_Use_Coupon_Count  Clicked_Coupon_Count      Revenue  
0                   0.0                   0.0    174.98174  
1                  14.0                  26.0  15686.84396  
2                   1.0                  11.0   1689.55594  
3                   3.0                   8.0   1467.43528  
4                   8.0                  17.0   2007.42900  
5                   5.0                   9.0    784.03320  
6                  18.0                  52.0   8059.28482  
7                   2.0                   9.0    595.56606  
8                  12.0                  44.0  11244.90066  
9                  10.0                  38.0   5865.09956

In [45]:
# Summary of custome_analysis
customer_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerID            1468 non-null   int64  
 1   Total_Spending        1468 non-null   float64
 2   Delivery_Charges      1468 non-null   float64
 3   Num_Transactions      1468 non-null   int64  
 4   Total_Quantity        1468 non-null   int64  
 5   Location              1468 non-null   object 
 6   Gender                1468 non-null   object 
 7   Tenure_Months         1468 non-null   int64  
 8   Used_Coupon_Count     1468 non-null   float64
 9   Not_Use_Coupon_Count  1468 non-null   float64
 10  Clicked_Coupon_Count  1468 non-null   float64
 11  Revenue               1468 non-null   float64
dtypes: float64(6), int64(4), object(2)
memory usage: 137.8+ KB


In [46]:
# Statistics summary of Revenue column in customer_analysis
customer_analysis['Revenue'].describe()

count     1468.000000
mean      3676.674895
std       5846.082106
min          6.990000
25%        783.974310
50%       2011.622610
75%       4495.056630
max      87200.896260
Name: Revenue, dtype: float64

In [47]:
customer_analysis.head()

CustomerID  Total_Spending  Delivery_Charges  Num_Transactions  \
0       12346           25.49            150.00                 2   
1       12347         3746.16            665.14                60   
2       12348          336.52            197.15                23   
3       12350         1312.40            127.88                17   
4       12356         1244.83            637.49                36   

   Total_Quantity    Location Gender  Tenure_Months  Used_Coupon_Count  \
0               3    New York      F             31                2.0   
1             342    New York      M             20               20.0   
2             209  California      M             39               11.0   
3              21  California      M             25                6.0   
4              56     Chicago      F             31               11.0   

   Not_Use_Coupon_Count  Clicked_Coupon_Count      Revenue  
0                   0.0                   0.0    174.98174  
1                  14.0                  26.0  15686.84396  
2                   1.0                  11.0   1689.55594  
3                   3.0                   8.0   1467.43528  
4                   8.0                  17.0   2007.42900

In [48]:
#customer_analysis.to_csv('customer_analysis.csv')

## Customer Purchased Product Analysis

In [49]:
# Extract unique Product_Description for each customer
cus_prod = sales.groupby('CustomerID')['Product_Description'].unique().reset_index()
# Extract unique Product_SKU for each customer
cus_sku = sales.groupby('CustomerID')['Product_SKU'].unique().reset_index()
# Extract unique Product_Category for each customer
prod_cat = sales.groupby('CustomerID')['Product_Category'].unique().reset_index()
# Extract unique Product Group for each customer
cus_group = sales.groupby('CustomerID')['Product_Group'].unique().reset_index()

# Merge cus_prod, cus_sku, prod_cat, and cus_group together
product_analysis = pd.merge(cus_prod, cus_sku, on='CustomerID')
product_analysis = pd.merge(product_analysis, prod_cat, on='CustomerID')
product_analysis = pd.merge(product_analysis, cus_group, on='CustomerID')

# Rename columns
product_analysis.columns = ['CustomerID', 'Purchased_Products', 'Products_SKUs', 'Product_Categories', 'Product_Group']

In [50]:
# Preview product_analysis
product_analysis.head()

CustomerID                                 Purchased_Products  \
0       12346  [Android Men's Engineer Short Sleeve Tee Charc...   
1       12347  [Four Color Retractable Pen, Red Spiral Google...   
2       12348  [26 oz Double Wall Insulated Bottle, Google St...   
3       12350  [Nest Learning Thermostat 3rd Gen-USA - Stainl...   
4       12356  [YouTube Men's Short Sleeve Hero Tee  White, R...   

                                       Products_SKUs  \
0                   [GGOEAAAJ080816, GGOEGOAR013099]   
1  [GGOEGOAQ020099, GGOEGOCT019199, GGOENEBJ07949...   
2  [GGOEGDHQ015399, GGOEGOAR013099, GGOEGBMJ01339...   
3  [GGOENEBJ079499, GGOENEBQ081599, GGOEGAEL09131...   
4  [GGOEYAAQ031717, GGOEGDWR015799, GGOEGADC05931...   

                                  Product_Categories  \
0                                  [Apparel, Office]   
1  [Office, Nest, Bags, Drinkware, Headgear, Appa...   
2  [Drinkware, Office, Bags, Lifestyle, Apparel, ...   
3                          [Nest, Apparel, Headgear]   
4  [Apparel, Drinkware, Nest, Bags, Office, Lifes...   

                          Product_Group  
0                     [Android, Google]  
1  [No-Brand, Google, YouTube, Android]  
2           [No-Brand, Google, YouTube]  
3                              [Google]  
4  [YouTube, No-Brand, Google, Android]

In [51]:
product_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CustomerID          1468 non-null   int64 
 1   Purchased_Products  1468 non-null   object
 2   Products_SKUs       1468 non-null   object
 3   Product_Categories  1468 non-null   object
 4   Product_Group       1468 non-null   object
dtypes: int64(1), object(4)
memory usage: 57.5+ KB


In [52]:
#product_analysis.to_csv('product_analysis.csv')